In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

In [ ]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse

Посмотрим для начала на тестовый и тренировочный датасеты

In [ ]:
train.info()
test.info()

Видим, что в тренировочном датасете на три столбца больше:

присутствует overall - оценка по пятибальной шкале присутствует reviewText - текст отзыва присутствует summary - краткое содержание отзыва присутствует rating - целевая переманная отсутствует Id Удалим столбцы overall, reviewText, summary из traina.

In [ ]:
train = train.drop(['reviewText', 'summary', 'overall'], axis=1)

Посмотрим на столбец verified

In [ ]:
print(train.verified)

In [ ]:
print('Количество пропусков: {},'.format(train.verified.isnull().sum()))
print('{},'.format(train.verified.describe()))

Пропусков нет. Заменим значения на 1 и 0.

In [ ]:
train['verified'] = np.where((train.verified == 'True'), 0, train.verified)
train['verified'] = np.where((train.verified == 'False' ), 1, train.verified) 
print(train.verified)


Посмотрим на столбцы reviewTime и unixReviewTime

In [ ]:
print(train.unixReviewTime)

In [ ]:
print('Количество пропусков: {},'.format(train.unixReviewTime.isnull().sum()))
print('{},'.format(train.unixReviewTime.describe()))

In [ ]:
print(train.reviewTime)

In [ ]:
print('Количество пропусков: {},'.format(train.reviewTime.isnull().sum()))
print('{},'.format(train.reviewTime.describe()))

В обоих столбцах содержится информация о дате отзыва, но в столбце reviewTime тип данных object нам не удобен. Удалим столбец.

In [ ]:
from datetime import datetime
tsmin = train.unixReviewTime.min()
ts25 = int(train.unixReviewTime.quantile(0.25))
ts50 = int(train.unixReviewTime.quantile(0.50))
ts75 = int(train.unixReviewTime.quantile(0.75))
tsmax = train.unixReviewTime.max()
print('Самый первый отзыв:', datetime.utcfromtimestamp(tsmin).strftime('%Y-%m-%d %H:%M:%S'))
print('25 квантиль:', datetime.utcfromtimestamp(ts25).strftime('%Y-%m-%d %H:%M:%S'))
print('50 квантиль:', datetime.utcfromtimestamp(ts50).strftime('%Y-%m-%d %H:%M:%S'))
print('75 квантиль:', datetime.utcfromtimestamp(ts75).strftime('%Y-%m-%d %H:%M:%S'))
print('Последний отзыв:', datetime.utcfromtimestamp(tsmax).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# Создадим 4 категории в зависимости от даты написания
def cat_date(x):
    if x <= ts25: x = '1'
    elif ts25 < x <= ts50: x = '2'
    elif ts50 < x <= ts75: x = '3'
    elif ts75 < x: x = '4'
    return x      

# Заменим значения в столбце на категории
train['unixReviewTime'] = train['unixReviewTime'].apply(lambda x: cat_date(x))

In [ ]:
train['unixReviewTime'] = train['unixReviewTime'].astype(int)

In [ ]:
train = train.drop(['reviewTime'], axis=1)

Посмотрим на столбец asin

In [ ]:
print(train.asin)

In [ ]:
print('Количество пропусков: {},'.format(train.asin.isnull().sum()))
print('{},'.format(train.asin.describe()))

In [ ]:
train.asin.value_counts()

заменим в этом же столбце код asin на количество его повторений.

In [ ]:
train['asin'] = train['asin'].map(train['asin'].value_counts())

In [ ]:
print(train.asin)

In [ ]:
train['asin'].describe()

In [ ]:
# Создадим 4 категории:
def cat_asin(x):
    if x <= 16: x = '1'
    elif 16 < x <= 52: x = '2'
    elif 52 < x <= 188: x = '3'
    elif 188 < x: x = '4'
    return x      

# Заменим значения в столбце на категории
train['asin'] = train['asin'].apply(lambda x: cat_asin(x))

In [ ]:
print(train.asin)

Посмотрим на столбец reviewerName

In [ ]:
print(train.reviewerName)

In [ ]:
print('Количество пропусков: {},'.format(train.reviewerName.isnull().sum()))
print('{},'.format(train.reviewerName.describe())

У нас есть столбец с похожей информацией - userid. Этот можно удалить.

In [ ]:
train = train.drop(['reviewerName'], axis=1)

Посмотрим на столбец vote

In [ ]:
print(train.vote)

In [ ]:
print('Количество пропусков: {},'.format(train.vote.isnull().sum()))
print('{},'.format(train.vote.describe()))

In [ ]:
train.vote.unique()

In [ ]:
# Удалим запятые
train['vote'] = train['vote'].astype('str')
train['vote'] = train['vote'].apply(lambda x: x.replace(',', ''))
# Заменим пропуски на 0
train['vote'] = train['vote'].apply(lambda x: x.replace('nan', '0'))
train.vote = train.vote.fillna(0)
# Приведем к int32
train['vote'] = train['vote'].astype('float')
train['vote'] = train['vote'].astype('int32')
# Посмотрим на результат
train.vote.unique()

In [ ]:
print('Количество пропусков: {},'.format(train.vote.isnull().sum()))
print('{},'.format(train.vote.describe()))

In [ ]:
# Создадим 4 категории:
def cat_vote(x):
    if x <= 16: x = '1'
    elif 16 < x <= 52: x = '2'
    elif 52 < x <= 188: x = '3'
    elif 188 < x: x = '4'
    return x      

# Заменим значения в столбце на категории
train['vote'] = train['vote'].apply(lambda x: cat_vote(x))

In [ ]:
train.vote.unique()

Посмотрим на столбец style

In [ ]:
print(train.style)

In [ ]:
print('Количество пропусков: {},'.format(train.style.isnull().sum()))
print('{},'.format(train.style.describe()))

In [ ]:
train = train.drop(['style'], axis=1)

Посмотрим на столбец image

In [ ]:
print(train.image)

In [ ]:
print('Количество пропусков: {},'.format(train.image.isnull().sum()))
print('{},'.format(train.image.describe()))

Столбец содержит картинки. Удаляем

In [ ]:
train = train.drop(['image'], axis=1)

Посмотрим на столбец userid

In [ ]:
print(train.userid)

In [ ]:
print('Количество пропусков: {},'.format(train.userid.isnull().sum()))
print('{},'.format(train.userid.describe()))

Пропусков нет, тип числовой, оставляем

Посмотрим на столбец itemid

In [ ]:
print(train.itemid)

In [ ]:
print('Количество пропусков: {},'.format(train.itemid.isnull().sum()))
print('{},'.format(train.itemid.describe()))

Пропусков нет, тип числовой, оставляем

Посмотрим что получилось в итоге

In [ ]:
train.info()

Наивная модель

In [ ]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)


In [ ]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [ ]:
NUM_THREADS = 4 #число потоков
NUM_COMPONENTS = 30 #число параметров вектора 
NUM_EPOCHS = 20 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [ ]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)


In [ ]:
sklearn.metrics.roc_auc_score(test_data.rating,preds)

Теперь построим модель, в которую добавим наши 4 признака

In [ ]:
# create train dataset

df = pd.DataFrame(train,  columns = ['userid', 'itemid', 'rating'])
df

In [ ]:
#create item features

features = pd.DataFrame(train,  columns = ['userid', 'verified', 'asin', 'unixReviewTime', 'vote'])
features

In [ ]:
uf = []
col = ['verified']*len(features.verified.unique()) + ['asin']*len(features.asin.unique()) + ['unixReviewTime']*len(features.unixReviewTime.unique()) + ['vote']*len(features['vote'].unique())
unique_verified = list(features.verified.unique()) + list(features.asin.unique()) + list(features.unixReviewTime.unique()) + list(features['vote'].unique())
#print('verified:', unique_verified)
for x,y in zip(col, unique_verified):
    res = str(x)+ ":" +str(y)
    uf.append(res)
    print(res)

In [ ]:
from lightfm.data import Dataset
# we call fit to supply userid, item id and user/item features
dataset1 = Dataset()
dataset1.fit(
        df['userid'].unique(), # all the users
        df['itemid'].unique(), # all the items
        user_features = uf
)

In [ ]:
# plugging in the interactions and their weights
(interactions, weights) = dataset1.build_interactions([(x[0], x[1], x[2]) for x in df.values ])

In [ ]:
def feature_colon_value(my_list):
    """
    Takes as input a list and prepends the columns names to respective values in the list.
    For example: if my_list = [1,1,3,2],
    resultant output = ['verified:1', 'asin:1', 'unixReviewTim:3', 'vote:2']
   
    """
    result = []
    ll = ['verified:','asin:', 'unixReviewTime:', 'vote:']
    aa = my_list
    for x,y in zip(ll,aa):
        res = str(x) +""+ str(y)
        result.append(res)
    return result

In [ ]:
ad_subset = features[["verified", 'asin','unixReviewTime', 'vote']] 
ad_list = [list(x) for x in ad_subset.values]
feature_list = []
for item in ad_list:
    feature_list.append(feature_colon_value(item))
    #print(feature_colon_value(item))
#print(f'Final output: {feature_list}')

In [ ]:
user_tuple = list(zip(features.userid, feature_list))
#user_tuple

In [ ]:
user_features = dataset1.build_user_features(user_tuple, normalize= False)
user_features.todense()

In [ ]:
user_features.todense()

In [ ]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset1.mapping()
#dataset1.mapping()

In [ ]:
model_fin = LightFM(loss='warp')
model_fin.fit(interactions, # spase matrix representing whether user u and item i interacted
      user_features= user_features, # we have built the sparse matrix above
      sample_weight= weights, # spase matrix representing how much value to give to user u and item i inetraction: i.e ratings
      epochs=10)

In [ ]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model_fin,
                      interactions,
                      user_features=user_features
                     ).mean()
print('Hybrid training set AUC: %s' % train_auc)

In [ ]:
# прогноз для существующего пользователя
user_x = user_id_map[88]
n_users, n_items = interactions.shape # no of users * no of items
model_fin.predict(user_x, np.arange(n_items)) # means predict for all

In [ ]:
# прогноз для нового пользователя
user_feature_list = ['verified:1', 'asin:2', 'unixReviewTime:4', 'vote:1']

In [ ]:
def format_newuser_input(user_feature_map, user_feature_list):
  #user_feature_map = user_feature_map  
  num_features = len(user_feature_list)
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [ ]:
new_user_features = format_newuser_input(user_feature_map, user_feature_list)

In [ ]:
model_fin.predict(0, np.arange(n_items), user_features=new_user_features) # Here 0 means pick the first row of the user_features sparse matrix

In [ ]:
# Создадим разреженную матрицу
ratings_coo2 = sparse.coo_matrix((train['rating'].astype(int),
                                 (train['userid'],
                                  train['itemid'],)))

# Обучим модель
model2 = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model2 = model2.fit(ratings_coo2, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [ ]:
preds = model2.predict(test.userid.values,
                      test.itemid.values)

In [ ]:
preds.min(), preds.max()

In [ ]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [ ]:
normalized_preds.min(), normalized_preds.max()

In [ ]:
submission['rating']= normalized_preds

In [ ]:
submission.to_csv('submission_log.csv', index=False)